# Regresión OLS Práctica
Katlyn Goeujon-Mackness <br>
31/31/2025 <br>
**Descripción:** Un espacio para practicar el método OLS de regresión lineal.

## Intervalos de confianza en regresión
* Descubrir cómo calcular pronósticos puntuales y sus intervalos de confianza en el contexto de la regresión lineal múltiple. 
* Utilizar la distribución T de Student y la desviación estándar de la regresión para evaluar la incertidumbre en las predicciones del modelo.


In [2]:
# Importar librerías
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Nota: Para este ejemplo asumirermos que los datos de entrenamiento son iguales a los datos de prueba.

In [3]:
Y = np.array([[3], [1], [8], [3], [5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [4]:
X = np.array([[1,3], [1,1], [1,5], [1,2], [1,4]])
X

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [5]:
XT_X = np.matmul(np.matrix.transpose(X), X)
XT_X

array([[ 5, 15],
       [15, 55]])

In [6]:
XT_X_inv = np.linalg.inv(XT_X)
XT_X_inv

array([[ 1.1, -0.3],
       [-0.3,  0.1]])

In [7]:
XT_Y = np.matmul(np.matrix.transpose(X), Y)
XT_Y

array([[20],
       [76]])

In [8]:
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[-0.8],
       [ 1.6]])

In [9]:
Y_pred = np.matmul(X, betas)
Y_pred

array([[4. ],
       [0.8],
       [7.2],
       [2.4],
       [5.6]])

In [10]:
Resid = Y - Y_pred
Resid

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [11]:
# Cálculo de TSS (Suma total de cuadrados)
TSS = np.matmul(np.matrix.transpose(Y), Y) - len(Y) * (Y.mean()**2)
TSS


array([[28.]])

In [12]:
# Cálculo de ESS (Suma explicada de cuadrados)
ESS = np.matmul(np.matmul(np.matrix.transpose(betas), np.matrix.transpose(X)), np.matmul(X, betas)) - len(Y)*(Y.mean()**2)
ESS

array([[25.6]])

In [13]:
# Cálculo de RSS (Residuales al cuadrado)
RSS = TSS - ESS
RSS

array([[2.4]])

In [14]:
# Cálculo del Coeficiente de Determinación R Cuadrada
RSq = float(1 - RSS / TSS)
RSq

0.9142857142857201

In [15]:
# Calcular la varianze del error
s_cuad = RSS/(len(Y) - X.shape[1])
s_cuad

array([[0.8]])

In [16]:
# Cálculo del Coeficiente de Determinación R Cuadrada Ajustada
RSqAj = 1 - (RSS / (X.shape[0] - X.shape[1])) / (TSS / (X.shape[0] - 1))
RSqAj

array([[0.88571429]])

In [17]:
# Desviación estándar del error
import math

s = math.sqrt(s_cuad)
s

0.8944271909998852

In [18]:
# Obtención del valor crítico de la t de Student
import scipy.stats

#Grados de libertad: n - (k+1)
grados_libertad = len(Y) - X.shape[1]
Confianza = 0.95
q = (1-Confianza) / 2
t_critico = abs(scipy.stats.t.ppf(q, df=grados_libertad))
t_critico

np.float64(3.182446305284263)

In [19]:
# Vector de valores particulares de X
f = np.array([[1], [7]])
f

array([[1],
       [7]])

In [20]:
Margen_error = t_critico * (s * (float(np.matmul(np.matmul(np.matrix.transpose(f), XT_X_inv),f)) ** 0.5))
Margen_error

np.float64(3.8189355663409854)

In [21]:
Pron_puntual = float(np.matmul(np.matrix.transpose(f), betas))
Pron_puntual

10.400000000000006

In [22]:
# Límites del intervalo de confianze
Lim_inferior = Pron_puntual - Margen_error
Lim_superior = Pron_puntual + Margen_error
print("Intervalo de confianza: (", Lim_inferior, ",", Lim_superior, ")")

Intervalo de confianza: ( 6.58106443365902 , 14.218935566340992 )


## Validación de supuestos de la regresión

In [23]:
import scipy
Resid

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [24]:
# Cálculo de la asimetría en residuales
skew = float(scipy.stats.skew(Resid, bias=True))
skew

-0.2886751345948135

In [25]:
# Cálculo de la curtosis de residuales
kurtosis = float(scipy.stats.kurtosis(Resid, fisher=False))
kurtosis

1.4499999999999993

In [26]:
Jarque_Bera = (len(Y)/6) * (skew ** 2 + (kurtosis - 3) ** 2 / 4)
Jarque_Bera

0.5699652777777785

In [27]:
Nivel_de_confianza = 0.95
scipy.stats.chi2.ppf(Nivel_de_confianza, df = 2)

np.float64(5.991464547107979)

Conclusion: Dado que JB no es mayor al nivel crítico, no podemos rechazar la hipótesis de existencia de normalidad en los residuales.

## Supuesto 2: Inexistencia de autocorrelación entre residuales

In [28]:
from statsmodels.formula.api import ols

In [29]:
y_df = pd.DataFrame(Y)
x_df = pd.DataFrame(X[:, 1:2])
x_df

,0
0,3
1,1
2,5
3,2
4,4


In [30]:
df = pd.concat([y_df, x_df.reindex(y_df.index)], axis = 1)
df.columns = ['Y', 'X1']
df

,Y,X1
0,3,3
1,1,1
2,8,5
3,3,2
4,5,4


In [31]:
# Ajuste de regresión lineal múltiple
model = ols('Y ~ X1', data = df).fit()

from statsmodels.stats.stattools import durbin_watson

# Prueba de Durbin-Watson
durbin_watson(model.resid)

np.float64(1.3666666666666656)

Conclusion: Dado que DW no es aprox. igual a 2, podemos pensar que existe autocorrelación entre los residuales.

## Supuesto 3: Homocedasticidad (Igual de varianzas)

In [32]:
ResidCuad = Resid ** 2
ResidCuad = pd.DataFrame(ResidCuad)
ResidCuad

,0
0,1.00
1,0.04
2,0.64
3,0.36
4,0.36


In [33]:
X1 = df.iloc[:,1]
X1_df = pd.DataFrame(X1)

X1Cuad = X1 ** 2
X1Cuad_df = pd.DataFrame(X1Cuad)

In [34]:
df_Aux = pd.concat([ResidCuad, X1_df.reindex(y_df.index), X1Cuad_df.reindex(y_df.index)], axis=1)
df_Aux.columns = ['Residual', 'X1', 'X1Cuad']
df_Aux

,Residual,X1,X1Cuad
0,1.00,3,9
1,0.04,1,1
2,0.64,5,25
3,0.36,2,4
4,0.36,4,16


In [35]:
# Ajuste de regresión lineal múltiple
modelAux = ols('Residual ~ X1 + X1Cuad', data=df_Aux).fit()
RSqAux = modelAux.rsquared
RSqAux

np.float64(0.5326278659612038)

In [36]:
Estadistico = len(Y) * RSqAux
Estadistico

np.float64(2.663139329806019)

In [37]:
Nivel_de_confianza = 0.95
scipy.stats.chi2.ppf(Nivel_de_confianza, df = 2)

np.float64(5.991464547107979)

Conclusión: Al no superar el valor crítico nuestro estadístico de prueba, no hay evidencia de Heterocedasticidad.

In [38]:
# Alternativa para la prueba de White

from statsmodels.stats.diagnostic import het_white

white_test = het_white(model.resid, model.model.exog)
print("Estadístico de prueba:", white_test[0])
print("Valor p:", white_test[1])

Estadístico de prueba: 2.66313932980599
Valor p: 0.26406244627058784


Conclusión: A un nivel de Alfa=5%, como tenemos un valor p superior a Alfa, no podemos rechazar la hipótesis nula.

## Supuesto 4: Inexistencia de Multicolinealidad
En este caso, no aplica realizarla ya que solo tenemos una vairbale regresora (X1). En models con más variables independientes, sí habría que realizarla.